<a id="top"></a>
# Export Notebook  

<hr><hr>

# Notebook Summary

The code in this notebook subsets a data cube, selects a specific set of variables, generates some additional data from those and then outputs that data into a GeoTIFF file. The goal is to be able to do external analyses of this data using other data analysis tools or GIS tools. The files would be reasonable in size, since we would restrict the region and parameters in the output.

<hr><hr>

# Index

* [Import Dependencies and Connect to the Data Cube](#import)
* [Choose Platforms and Products](#plat_prod)
* [Get the Extents of the Cube](#extents)
* [Define the Extents of the Analysis](#define_extents)
* [Load Data from the Datacube](#retrieve_data)
* [Derive Products](#derive_products)
* [Combine Data](#combine_data)
* [Export Data](#export)
    * [Export to GeoTIFF](#export_geotiff)
    * [Export to NetCDF](#export_netcdf)
    
<hr><hr>

## <span id="import">Import Dependencies and Connect to the Data Cube [&#9652;](#top)</span> 

In [ ]:
import xarray as xr  
import numpy as np
import datacube
from utils.data_cube_utilities.data_access_api import DataAccessApi  

In [ ]:
api = DataAccessApi(config = '/home/localuser/.datacube.conf')
dc = api.dc

## <span id="plat_prod">Choose Platforms and Products [&#9652;](#top)</span>

**List available products for each platform**

In [ ]:
list_of_products = dc.list_products()
netCDF_products = list_of_products[list_of_products['format'] == 'NetCDF']
netCDF_products

**Choose product**

In [ ]:
# platform = "LANDSAT_7"
# product = "ls7_ledaps_vietnam"

platform = "LANDSAT_8"
product = "ls8_lasrc_vietnam"

## <span id="extents">Get the Extents of the Cube [&#9652;](#top)</span>

In [ ]:
# Get product extents
prod_extents = api.get_query_metadata(platform=platform, product=product)#.get_full_dataset_extent(platform = platform, product = product)

latitude_extents = prod_extents['lat_extents']
print("Lat bounds:", latitude_extents)
longitude_extents = prod_extents['lon_extents']
print("Lon bounds:", longitude_extents)
time_extents = list(map(lambda time: time.strftime('%Y-%m-%d'), prod_extents['time_extents']))
print("Time bounds:", time_extents)

In [ ]:
## The code below renders a map that can be used to orient yourself with the region.
from utils.data_cube_utilities.dc_display_map import display_map
display_map(latitude = latitude_extents, longitude = longitude_extents)

## <span id="define_extents">Define the Extents of the Analysis [&#9652;](#top)</span>

In [ ]:
######### Vietnam - Buan Tua Srah Lake ################## 
longitude_extents = (108.02, 108.15)
latitude_extents  = (12.18 , 12.30)

time_extents = ('2015-01-01', '2016-01-01')

In [ ]:
from utils.data_cube_utilities.dc_display_map import display_map
display_map(latitude = latitude_extents, longitude = longitude_extents)

## <span id="retrieve_data">Load Data from the Data Cube [&#9652;](#top)</span>

In [ ]:
landsat_dataset = dc.load(latitude = latitude_extents,
                          longitude = longitude_extents,
                          platform = platform,
                          time = time_extents,
                          product = product,
                          measurements = ['red', 'green', 'blue', 'nir', 'swir1', 'swir2', 'pixel_qa']) 

In [ ]:
landsat_dataset

## <span id="derive_products">Derive Products [&#9652;](#top)</span> 

> ### Masks

In [ ]:
from utils.data_cube_utilities.clean_mask import landsat_qa_clean_mask

clear_xarray = landsat_qa_clean_mask(landsat_dataset, platform, cover_types=['clear'])
water_xarray = landsat_qa_clean_mask(landsat_dataset, platform, cover_types=['water'])
shadow_xarray = landsat_qa_clean_mask(landsat_dataset, platform, cover_types=['shadow'])

clean_xarray = xr.ufuncs.logical_or(clear_xarray , water_xarray).rename("clean_mask")

> ### Water Classification

In [ ]:
from utils.data_cube_utilities.dc_water_classifier import wofs_classify

water_classification = wofs_classify(landsat_dataset,
                                     clean_mask = clean_xarray.values, 
                                     mosaic = False) 

In [ ]:
wofs_xarray = water_classification.wofs

> ###  Normalized Indices  

In [ ]:
def NDVI(dataset):
    return ((dataset.nir - dataset.red)/(dataset.nir + dataset.red)).rename("NDVI")

In [ ]:
def NDWI(dataset):
    return ((dataset.green - dataset.nir)/(dataset.green + dataset.nir)).rename("NDWI")

In [ ]:
def NDBI(dataset):
        return ((dataset.swir2 - dataset.nir)/(dataset.swir2 + dataset.nir)).rename("NDBI")

In [ ]:
ndbi_xarray = NDBI(landsat_dataset)  # Urbanization - Reds
ndvi_xarray = NDVI(landsat_dataset)  # Dense Vegetation - Greens
ndwi_xarray = NDWI(landsat_dataset)  # High Concentrations of Water - Blues  

>### TSM  

In [ ]:
from utils.data_cube_utilities.dc_water_quality import tsm

tsm_xarray = tsm(landsat_dataset, clean_mask = wofs_xarray.values.astype(bool) ).tsm

> ### EVI  

In [ ]:
def EVI(dataset, c1 = None, c2 = None, L = None):
        return ((dataset.nir - dataset.red)/((dataset.nir  + (c1 * dataset.red) - (c2 *dataset.blue) + L))).rename("EVI")

In [ ]:
evi_xarray = EVI(landsat_dataset, c1 = 6, c2 = 7.5, L = 1 )

## <span id="combine_data">Combine Data [&#9652;](#top)</span>  

In [ ]:
combined_dataset = xr.merge([landsat_dataset,
          ## <span id="combine_data">Combine Data [&#9652;](#top)</span>  clean_xarray,
          clear_xarray,
          water_xarray,
          shadow_xarray,
          evi_xarray,
          ndbi_xarray,
          ndvi_xarray,
          ndwi_xarray,
          wofs_xarray,
          tsm_xarray])

# Copy original crs to merged dataset 
combined_dataset = combined_dataset.assign_attrs(landsat_dataset.attrs)

combined_dataset

## <span id="export">Export Data [&#9652;](#top)</span>  

### <span id="export_geotiff">Export to GeoTIFF [&#9652;](#top)</span>  

Export each acquisition as a GeoTIFF.

In [ ]:
from utils.data_cube_utilities.import_export import export_xarray_to_geotiff

# Ensure the output directory exists before writing to it.
if platform == 'LANDSAT_7':
    !mkdir -p output/geotiffs/landsat7
else:
    !mkdir -p output/geotiffs/landsat8

output_path = "output/geotiffs/landsat{0}/landsat{0}".format(7 if platform=='LANDSAT_7' else 8)

export_xarray_to_geotiff(combined_dataset, output_path)

Check to see what files were exported. The size of these files is also shown.

In [ ]:
if platform == 'LANDSAT_7':
    !ls -lah output/geotiffs/landsat7/*.tif
else:
    !ls -lah output/geotiffs/landsat8/*.tif

Sanity check using `gdalinfo` to make sure that all of our bands exist    .

In [ ]:
if platform == 'LANDSAT_7':
    !gdalinfo output/geotiffs/landsat7/landsat7_2015_01_09_03_06_13.tif
else:
    !gdalinfo output/geotiffs/landsat8/landsat8_2015_01_01_03_07_41.tif

Zip all GeoTIFFs.

In [ ]:
if platform == 'LANDSAT_7':
    !tar -cvzf output/geotiffs/landsat7/landsat_7.tar.gz output/geotiffs/landsat7/*.tif
else:
    !tar -cvzf output/geotiffs/landsat8/landsat_8.tar.gz output/geotiffs/landsat8/*.tif

### <span id="export_netcdf">Export to NetCDF [&#9652;](#top)</span>  

Export all acquisitions together as a single NetCDF.

In [ ]:
from utils.data_cube_utilities.import_export import export_xarray_to_netcdf

# Ensure the output directory exists before writing to it.
if platform == 'LANDSAT_7':
    !mkdir -p output/netcdfs/landsat7
else:
    !mkdir -p output/netcdfs/landsat8

output_file_path = "output/netcdfs/landsat{0}/ls{0}_netcdf_example.nc".format(7 if platform=='LANDSAT_7' else 8)
# Remove the file if it exists to avoid an error.
import os
if os.path.isfile(output_file_path):
    os.remove(output_file_path)
export_xarray_to_netcdf(combined_dataset, output_file_path)

Sanity check using `gdalinfo` to make sure that all of our bands exist  .  

In [ ]:
if platform == 'LANDSAT_7':
    !gdalinfo output/netcdfs/landsat7/ls7_netcdf_example.nc
else:
    !gdalinfo output/netcdfs/landsat8/ls8_netcdf_example.nc